# Setup

Install all needed dependencies.

In [1]:
!pip install --quiet --upgrade transformers sentence-transformers


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# Install the YAML magic
!pip install --quiet yamlmagic
%load_ext yamlmagic


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


Configuration and used models.

In [2]:
%%yaml parameters

# Model
embedder_model_name_or_path: ibm-granite/granite-embedding-30m-english
generator_model_name_or_path: ibm-granite/granite-3.2-2b-instruct

<IPython.core.display.Javascript object>

#  Simple RAG

Prepare chunk database. In this case database is represented by a list.

In [3]:
import urllib.request

link = "https://huggingface.co/ngxson/demo_simple_rag_py/raw/main/cat-facts.txt"
dataset = []

# Retrieve knowledge from provided link, use every line as a separate chunk.
for line in urllib.request.urlopen(link):
  dataset.append(line.decode('utf-8'))

print(f'Loaded {len(dataset)} entries')

Loaded 150 entries


**Specify user query here**

In [4]:
input_query = "tell me about cat mummies"

Encode user query and chunks into embeddings (vector representations). Use semantic_search to find 5 chunks which are most similar to the query.

In [5]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import semantic_search

embedder = SentenceTransformer(parameters['embedder_model_name_or_path'])

query_embedding = embedder.encode(input_query, convert_to_tensor=True)
dataset_embedding = embedder.encode(dataset, convert_to_tensor=True)

# Get 5 chunk embeddings which are most similar to the query embedding.
# semantic_search returns list of dictionaries with embedding index (corpus_id) and similarity score.
retrieved_knowledge = semantic_search(query_embeddings=query_embedding, corpus_embeddings=dataset_embedding, top_k=5)

print('Retrieved knowledge:')
for corpus in retrieved_knowledge[0]:
  print(f' - (similarity: {corpus["score"]:.2f}) {dataset[corpus["corpus_id"]]}')

Retrieved knowledge:
 - (similarity: 0.85) In ancient Egypt, mummies were made of cats, and embalmed mice were placed with them in their tombs. In one ancient city, over 300,000 cat mummies were found.

 - (similarity: 0.80) In 1888, more than 300,000 mummified cats were found an Egyptian cemetery. They were stripped of their wrappings and carted off to be used by farmers in England and the U.S. for fertilizer.

 - (similarity: 0.79) When a family cat died in ancient Egypt, family members would mourn by shaving off their eyebrows. They also held elaborate funerals during which they drank wine and beat their breasts. The cat was embalmed with a sculpted wooden mask and the tiny mummy was placed in the family tomb or in a pet cemetery with tiny mummies of mice.

 - (similarity: 0.75) Mohammed loved cats and reportedly his favorite cat, Muezza, was a tabby. Legend says that tabby cats have an “M” for Mohammed on top of their heads because Mohammad would often rest his hand on the cat’s he

Prepare inference pipeline using transformers

In [6]:
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained(parameters['generator_model_name_or_path'])
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

pipeline = transformers.pipeline("text-generation", model=parameters['generator_model_name_or_path'], tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Run inference using provided user prompt and system prompt containing knowledge chunks.

In [7]:
# Construct system prompt for inference providing retrieved chunks as context.
instruction_prompt = f'''You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
{''.join([f' - {dataset[corpus["corpus_id"]]}' for corpus in retrieved_knowledge[0]])}
'''

messages = [
    {
        "role": "system",
        "content": instruction_prompt,
    },
    {
        "role": "user",
        "content": input_query,
    }
]

outputs = pipeline(messages, max_new_tokens=1024)

Print result.

In [8]:
from IPython.display import display, Markdown

output = ""
for turn in outputs:
    for item in turn["generated_text"]:
        output += f"# {item['role']}\n\n{item['content']}\n\n"

display(Markdown(output))

# system

You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
 - In ancient Egypt, mummies were made of cats, and embalmed mice were placed with them in their tombs. In one ancient city, over 300,000 cat mummies were found.
 - In 1888, more than 300,000 mummified cats were found an Egyptian cemetery. They were stripped of their wrappings and carted off to be used by farmers in England and the U.S. for fertilizer.
 - When a family cat died in ancient Egypt, family members would mourn by shaving off their eyebrows. They also held elaborate funerals during which they drank wine and beat their breasts. The cat was embalmed with a sculpted wooden mask and the tiny mummy was placed in the family tomb or in a pet cemetery with tiny mummies of mice.
 - Mohammed loved cats and reportedly his favorite cat, Muezza, was a tabby. Legend says that tabby cats have an “M” for Mohammed on top of their heads because Mohammad would often rest his hand on the cat’s head.
 - Cats are subject to gum disease and to dental caries. They should have their teeth cleaned by the vet or the cat dentist once a year.



# user

tell me about cat mummies

# assistant

In ancient Egypt, cat mummies were a significant part of their funeral rituals. These mummies were made from actual cats, not just representations. The process involved embalming the cat, often with a sculpted wooden mask, and then preserving the body. 

The practice of mumming cats was particularly common for family pets. When a family cat died, the family would mourn by shaving off their eyebrows and holding elaborate funerals. During these ceremonies, they would drink wine and beat their breasts, symbolizing grief. The embalmed cat, along with a tiny mummy of a mouse (to represent the cat's prey), was then placed in the family tomb or in a pet cemetery.

One notable instance occurred in 1888 when over 300,000 cat mummies were discovered in an Egyptian cemetery. These were stripped of their wrappings and exported to England and the U.S. for use as fertilizer. This event highlights the historical significance and practical use of cat mummies in ancient Egypt.

Interestingly, the practice of mumming cats was also influenced by cultural figures. For example, Mohammed, the founder of Islam, was known to love cats, with his favorite, Muezza, being a tabby. According to legend, tabby cats bear an "M" on their heads, symbolizing Mohammed's affection for them.

It's worth noting that while these historical practices are fascinating, modern veterinary care is crucial for cat health. Cats, like all pets, can suffer from dental issues such as gum disease and dental caries. Regular dental check-ups and cleanings by a vet or a cat dentist are recommended to maintain their oral health.

